### Convolutional Neural Network
 A convolutional neural network (CNN, or ConvNet) is a class of deep neural networks, most commonly applied to analyze visual images.
One way that might improve results in fully conneceted layers is by adding more hidden layers or making the network to be deep. But this can cause problems such as vanishing gradient. In order to fix this problem in a model, the ReLU activation functions can be used. Another problem that is associted with deep fully connected networks is that the trainable parameters (weights) in the model can grow rapidly. This might result to slow training or the training can never happen, and also might lead to overfitting of the model. We can solve this problem by using the convolution neural network. How??

Convolution neural network exploits the correlation between adjecent inputs in the images. For example, It is likely that the pixels aclose to the cat's eyes are more likely to be connected with nearby pixels which show the cats nose-rather than the pixels on the other side of the image that represents the dog's nose. Therefore, not every other node in the network needs to be connecetd to every other node in the next layer. This helps in reducng the number of weights or parameters required to be trained in the model.

<img src="cnn1.png">

[Diagram source](https://www.udemy.com/course/deep-learning-machine-learning-practical/)


**How does the CNN works?**

The convolution neural network is composed of the following layers in which each layer perform specific tasks:

**Convolution layer**

- Uses image kernel matrix also kwon as filter.
- It performs convolution operation (dot product of the original pixel values with the weights defined in the filter.
- The kernel is scanned over a given image and does feauture extraction to select most important pixels of that image.
- The kernel apply effects such as blurring and sharpening.

**How does convolutional operation help?**

The convolutional neural network reduce the number of parameters which need to be trained by using the following properties.
1. It provides sparse connections – not every node in the first / input layer is connected to every node in the second layer. This is contrary to fully connected neural networks, where every node is connected to every other in the following layer.
2.   Constant filter parameters – each filter has constant parameters. In other words, as the filter moves around the image, the same weights are applied to each 2 x 2 set of nodes. Each filter, as such, can be trained to perform a certain specific transformation of the input space. Therefore, each filter has a certain set of weights that are applied for each convolution operation – this reduces the number of parameters.

<img src="conv.png">
[Diagram source](https://www.udemy.com/course/deep-learning-machine-learning-practical/)

**Activation layer**
- The activation function that is used in CNN is the non linear RELU activation function.
- ReLU is used to add non-linearity in the feature map.
- It enhances the sparsity or how scattered the feature map is.
- The advantage of ReLU is that, its gradinet does not vanish as we increase the input as compared to sigmoid function.
- ReLU function is represented as; f(x)= max(0,x), this implies that, it takes in input x and return x if x is positive and returns 0 if x is negative.

The following is an illustraion of how ReLU function works.

<img src="Relu.png">

[Diagram source](https://www.udemy.com/course/deep-learning-machine-learning-practical/)

**Pooling or down-sampling layer**
- This layer reduces the dimensionality or size of the feature map or the number of trainable parameters in the model.
- This helps is improving the computational efficency by allowing the network to train much faster, focusing on the most importatnt information.
- It also helps in generalizing the model hence avoiding overfitting.
- There various kinds of pooling but the nost commonly used is the maximum pooling.
- The maximum pooling works by retaining the maximum value in a given sample size of a feature map.

The following is a pooling diagram which illustrate how maximum pooling works.

<img src="Pool.png">

[Diagram source](https://www.udemy.com/course/deep-learning-machine-learning-practical/)

- Strides and down-sampling

In the pooling diagram above, we can see that the pooling window in blue color which supposed to be shifted to the right each time by 2 places, both in x and y direction. In other words, the stride is specified as [2, 2]. One important thing to notice is that, if during pooling the stride is greater than 1, then the output size will be reduced. As can be observed above, the 4 x 4 input is reduced to a 2 x 2 output. This is a good thing – it is called down-sampling, and it reduces the number of trainable parameters in the model.

- Padding

Padding is adding an extra column and row to the input image for example the 4 $\times$ 4 input, this helps is giving the effective size of the pooling space. It helps in ensuring that the pooling windo can correctly operate with a given stride.  Padding may or may not be considered when constructing a convolutional neural network in PyTorch.


**Fully connected layer**
- The last layer in a CNN is the fully connected layer. 
- This layer performs interpretation of the results and gives an output of the classified result. 
- In order to attach this fully connected layer to the network, the dimensions of the output of the CNN need to be flattened.

The diagram below show a flatten pooled matrix and the fully connected layer attached to the CNN.


<img src="flat.png">

[Diagram source](https://www.udemy.com/course/deep-learning-machine-learning-practical/)

**How to improve accuracy of a CNN?**

- The accuary of the model can be improved by increasing the number of filters or by adding a dropout.
- Dropout refers to dropping out units/neurons in the network.
- We drop the neurons because if they are too many they might develop co-dependency amongst each other during training.
- Dropout is a regularisation technique for reducing overfitting in neural network.


**Implementing CNN using pytorch**

In [1]:
import random
import PIL
import imageio
import matplotlib.pyplot as plt
import numpy as np
import skimage.transform
import torch
import torch.nn as nn
import torch.utils.data
import torchvision
from torchvision import datasets, transforms
from IPython import display


In [2]:
%matplotlib inline

def show(img):
    """Show PyTorch tensor img as an image in matplotlib."""
    npimg = img.cpu().detach().numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    plt.grid(False)
    plt.gca().axis('off')

def display_thumb(img):
  display.display(transforms.Resize(128)(img))

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
# Load the training and test dataset.
mnist_train = datasets.MNIST('/tmp/mnist', train=True, download=True)
mnist_test = datasets.MNIST('/tmp/mnist', train=False, download=True)

In [5]:
# Show a random image and the corresponding target.
img, target = mnist_train[0]
print('Label of image:', mnist_train.classes[target])
img

Label of image: 5 - five


In [6]:
# This ensures the MNIST dataset produces PyTorch tensors.
mnist_train.transform = transforms.ToTensor()
mnist_test.transform = transforms.ToTensor()

# Size of the batches the data loader will produce.
batch_size = 64

# This creates the dataloaders.
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

In [7]:
next(iter(train_loader))[0].shape

torch.Size([64, 1, 28, 28])

In [11]:
class ConvolutionalNetwork(nn.Module):
  """Simple convolutional network."""
  
  def __init__(self, input_size, num_classes, in_channels=1, num_neurons=50):
      super(ConvolutionalNetwork, self).__init__()
      
      self.conv_network1 = nn.Sequential(nn.Conv2d(in_channels, num_neurons,
                                kernel_size=5),nn.ReLU(),nn.MaxPool2d(kernel_size=2,stride=2))
                                        
        
                        
      self.conv_network2 = nn.Sequential(nn.Conv2d( num_neurons, 2*num_neurons,
                                kernel_size=5),nn.ReLU(),nn.MaxPool2d(kernel_size=2,stride=2))
                                        
      self.drop_out = nn.Dropout()                          
      self.linear1= nn.Linear(2*num_neurons*4*4,50)
      self.linear2 = nn.Linear(50,num_classes)
      
  def forward(self, x):

      x = self.conv_network1(x)
      x = self.conv_network2(x)
      
      x = x.view(x.size(0), -1)
      x = self.drop_out(x)
      x = self.linear1(x)
      x = self.linear2(x)
      return x





In [12]:
def train(model, criterion, data_loader, optimizer, num_epochs):
    """Simple training loop for a PyTorch model."""
    
    # Make sure model is in training mode.
    model.train()
    # Move model to the device.
    model.to(device)
    
    # Exponential moving average of the loss.
    ema_loss = None
    
    # Loop over epochs.
    for epoch in range(num_epochs):
      
      # Loop over data.
      for batch_idx, (data, target) in enumerate(data_loader):
          data = data.to(device)
          target = target.to(device)
        
          # Forward pass.
          output = model(data)
          loss = criterion(output, target)
          
          # Backward pass.
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          
          if ema_loss is None:
            ema_loss = loss.item()
          else:
            ema_loss += (loss.item() - ema_loss) * 0.01 
          
          # Print out progress.
          if batch_idx % 500 == 0:
              print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), 
                    len(data_loader.dataset),
                    100. * batch_idx / len(data_loader), ema_loss),
              )

In [13]:
def test(model, data_loader):
    """Measures the accuracy of a model on a data set."""
    
    # Make sure the model is in evaluation mode.
    model.eval()
    correct = 0

    # We do not need to maintain intermediate activations while testing.
    with torch.no_grad():
      
        # Loop over test data.
        for data, target in data_loader:
          
            # Forward pass.
            output = model(data.to(device))
            
            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)
            
            # Count number of correct predictions.
            correct += pred.cpu().eq(target.view_as(pred)).sum().item()

    # Print test accuracy.
    print('Accuracy: {}/{} ({:.0f}%)\n'.format(
          correct, 
          len(data_loader.dataset),
          100. * correct / len(data_loader.dataset)),
    )  

In [14]:
# Create and train convolutional network.
conv_model = ConvolutionalNetwork(28, 10)

#criterion
criterion = nn.CrossEntropyLoss()

# Make sure you optimize conv_model rather than model.
optimizer =torch.optim.SGD(conv_model.parameters(), lr=0.01)

#training the model
train(conv_model, criterion, train_loader, optimizer, num_epochs=5)

#testing the model
test(conv_model, test_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.329809
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.747553
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.348115
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.246460
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.195598
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.163108
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.158312
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.135728
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.119801
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.112035
Accuracy: 9768/10000 (98%)

